Author: Noah Brenowitz

In this notebook, I demonstrate restarting FV3GFS from data sent by spencer. The main complications are 
1. The surface data needs to be coarse-grained.
2. 

In [ ]:
%matplotlib inline

In [ ]:
cd ../

In [ ]:
from src.fv3 import *
from os.path import join

# Running the analysis

Download the C96 restart data if it doesn't exist

In [ ]:
%%bash

outputdir=2019-09-30-C3072-to-C96-restart


if [[ ! -d data/raw/$outputdir ]]
then
    file=2019-09-30-C3072-X-SHiELD-to-C96-restart.tar.gz
    gsutil cp gs://vcm-ml-data/$file .
    tar xzf $file
    mv $outputdir data/raw
fi

Download the rundata for 

In [ ]:
%%bash

[[ -e experiments/2019-10-02-restart_C48_from_C3072_rundir/ ]] && exit 0

mkdir -p experiments

file=2019-10-02-restart_C48_from_C3072_rundir.tar
gsutil cp gs://vcm-ml-data/$file .

    (cd  experiments 
    
     tar xf ../$file
     
    )
rm -rf $file

In [ ]:
sfc_data_dir = 'data/raw/2019-09-30-C3072-to-C96-restart/'
template_dir = 'experiments/2019-10-02-restart_C48_from_C3072_rundir/restart_C48_from_C3072_nosfc/'

In [ ]:
sfc_data = coarsen_sfc_data_in_directory(sfc_data_dir, method='median')

In [ ]:
sfc_data.slmsk.plot(col='tiles')

I fixed the integer valued sea land mask.

In [ ]:
rundir = make_experiment("2019-10-03-fixIceBug", sfc_data, namelist_path="assets/restart_c48.nml", template_dir=template_dir)
run_experiment(rundir)

In [ ]:
!tail -f {rundir}/rundir/fv3.out | grep DATE

# Analyzing the output of the run

In [ ]:
import xarray as xr

In [ ]:
data = xr.open_mfdataset("experiments/2019-10-03-fixIceBug/rundir/atmos_4xdaily.tile*.nc", concat_dim='tiles')

In [ ]:
data.Q925.plot(row='tiles', col='time')